# Data Preprocessing 

This step involved cheaning data. Followed by combining databases containing treasury rate (r) and the option data. Calculating historial and calibrated volatility, tau (τ), and other parameters

In [ ]:
import tensorflow as tf
from tensorflow.keras import Input, layers, Model, utils, initializers, losses, optimizers, Sequential, callbacks, backend
import tensorflow.keras as K

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from scipy.stats import norm
import scipy.stats as measures

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline 
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.family'] = 'sans-serif'

In [ ]:
plt.rc('font', size=24)          # controls default text sizes
plt.rc('axes', titlesize=28)     # fontsize of the axes title
plt.rc('axes', labelsize=28)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=24)    # fontsize of the tick labels
plt.rc('ytick', labelsize=24)    # fontsize of the tick labels
plt.rc('legend', fontsize=24)    # legend fontsize
plt.rc('figure', titlesize=28)  # fontsize of the figure title

In [ ]:
from datetime import date

In [ ]:
import math

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


## Utility Functions

In [ ]:
months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

In [ ]:
def get_interest_rate(row):
  dur = row['date_ndiff']
  if dur <= 91:
        select = '3M'
  elif dur <= 182:
      select = '6M'
  elif dur <= 365:
      select = '1Y'
  elif dur <= 730:
      select = '2Y'
  elif dur <= 1095:
      select = '3Y'
  elif dur <= 1461:
      select = '4Y'
  elif dur <= 1460:
      select = '5Y'
  elif dur <= 1826:
      select = '6Y'
  elif dur <= 2191:
      select = '7Y'
  elif dur <= 2556:
      select = '8Y'
  elif dur <= 2922:
      select = '9Y'
  elif dur <= 3287:
      select = '10Y'
  elif dur <= 3652:
      select = '11Y'
  elif dur <= 3975:
      select = '12Y'
  elif dur <= 4018:
      select = '13Y'
  elif dur <= 4363:
      select = '14Y'
  elif dur <= 4748:
      select = '15Y'
  elif dur <= 6209:
      select = '19Y'
  elif dur <= 8035:
      select = '24Y'
  else:
      select = '30Y'
  return interest_rate[select][len(interest_rate.index)-1]

In [ ]:
def date_format_change(row):
  d1 = row['Date']
  month, day, year = d1[0:3], d1[4:6], d1[8:]
  d2 = day+'-'+month+'-'+year
  return d2

In [ ]:
def get_interest_rate(row):
  dur = row['date_ndiff']
  if dur <= 91:
        select = '3mo'
  elif dur <= 182:
      select = '6mo'
  elif dur <= 365:
      select = '1yr'
  elif dur <= 730:
      select = '2yr'
  elif dur <= 1095:
      select = '3yr'
  elif dur <= 1461:
      select = '4yr'
  else:
      select = '5yr'
  return row[select]

In [ ]:
def date_diff(row):
    start = str(row['Date'])
    end = str(row['Expiry'])
    s_day, s_month, s_year = start[:2], start[3:6], start[7:]
    e_day, e_month, e_year = end[:2], end[3:6], end[7:]
    s_month = months[s_month]
    e_month = months[e_month]
    
    return date(int(e_year), e_month, int(e_day)) - date(int(s_year), s_month, int(s_day))

def numeric_date(row):
    return int(str(row['date_diff']).split()[0])

In [ ]:
def historical_volitlity(returns):
    mean = np.mean(returns)
    n = len(returns) 
    return math.sqrt(np.sum((returns-mean)**2)/(n-1))

In [ ]:
def calibrated_volitlity(returns):
    mean = np.mean(returns)
    n = len(returns) 
    return np.std(returns-mean)

In [ ]:
def delta_t(row):
    end = str(row['Date'])
    start = str(row['delta_t'])
    # print(start))
    if start == "nan":
        return np.nan
    start = str(start)
    s_day, s_month, s_year = start[:2], start[3:6], start[7:]
    e_day, e_month, e_year = end[:2], end[3:6], end[7:]
    s_month = months[s_month]
    e_month = months[e_month]
    differ = date(int(e_year), e_month, int(e_day)) - date(int(s_year), s_month, int(s_day))
   
    return int(str(differ).split()[0]) / 365

## Dropping columns
This step involved removing unnecessary columns

In [ ]:
df = pd.read_csv("NIFTY_PE.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# df = pd.read_csv('NIFTY_PE.csv',engine='python',encoding='UTF-8')

In [ ]:
df = df.dropna()

In [ ]:
df.tail()

,Symbol,Date,Expiry,Option Type,Strike Price,Open,High,Low,Close,LTP,Settle Price,No. of contracts,Turnover in Lacs,Premium Turnover in Lacs,Open Int,Change in OI,Underlying Value
62073,NIFTY,28-Jan-2015,29-Jan-2015,PE,7400,0.25,0.6,0.25,0.45,0.40,0.45,1113,2059.18,-,360800,-17875,8914.3
62074,NIFTY,28-Jan-2015,29-Jan-2015,PE,6850,0.00,0.0,0.00,0.95,0.00,0.00,0,0.00,-,0,0,8914.3
62075,NIFTY,28-Jan-2015,29-Jan-2015,PE,6900,0.20,0.3,0.20,0.25,0.25,0.25,10,17.25,-,3925,-75,8914.3
62076,NIFTY,28-Jan-2015,29-Jan-2015,PE,6950,0.00,0.0,0.00,1.75,0.00,0.00,0,0.00,-,0,0,8914.3
62077,NIFTY,28-Jan-2015,29-Jan-2015,PE,7000,0.60,0.6,0.20,0.40,0.35,0.40,3637,6365.15,-,169725,-26100,8914.3


In [ ]:
df = df.drop(['Symbol','Option Type','Open','High','Low','LTP','Settle Price','No. of contracts','Turnover in Lacs', 'Premium Turnover in Lacs','Open Int','Change in OI'], axis=1)

In [ ]:
df.head()

,Date,Expiry,Strike Price,Close,Underlying Value
0,03-Nov-2014,27-Nov-2014,8950,1014.55,8324.15
1,03-Nov-2014,27-Nov-2014,9000,629.35,8324.15
2,03-Nov-2014,27-Nov-2014,8700,340.45,8324.15
3,03-Nov-2014,27-Nov-2014,8750,645.15,8324.15
4,03-Nov-2014,27-Nov-2014,8800,411.70,8324.15


## Calculate days to expiry

In [ ]:
df['date_diff'] = df.apply(lambda row: date_diff(row), axis=1)

In [ ]:
df['date_ndiff'] = df.apply(lambda row: numeric_date(row), axis=1)
df = df.drop('date_diff', axis=1)

In [ ]:
df = df.drop(['Expiry'], axis=1)

## Underlying
This step invloved separating the underlying asset price for calculating features like log returns, historical volatility, calibrated volatility, delta_t

In [ ]:
underlying = df[['Date', 'Underlying Value']]

In [ ]:
underlying = underlying.drop_duplicates()

In [ ]:
underlying = underlying.rename(columns={'Underlying Value': 'stock_price'})

In [ ]:
underlying['stock_price'] = pd.to_numeric(underlying['stock_price'], errors='coerce')

In [ ]:
underlying = underlying.dropna()

In [ ]:
underlying['log_ret'] = np.log(underlying['stock_price']) - np.log(underlying['stock_price'].shift(1))

In [ ]:
underlying['sigma_20'] = underlying['log_ret'].rolling(19).apply(lambda x: historical_volitlity(x))

In [ ]:
underlying['cal_sig_20'] = underlying['log_ret'].rolling(19).apply(lambda x: calibrated_volitlity(x))

In [ ]:
underlying['delta_t'] = underlying['Date'].shift(19)

In [ ]:
underlying['delta_t'] = underlying.apply(lambda x: delta_t(x), axis=1)

In [ ]:
underlying['cal_sig_20'] = underlying.apply(lambda row: row['cal_sig_20']/math.sqrt(row['delta_t']), axis=1) 

In [ ]:
underlying = underlying.dropna()

In [ ]:
underlying.head()

,Date,stock_price,log_ret,sigma_20,cal_sig_20,delta_t
20229,02-Dec-2014,8524.70,-0.003653,0.004723,0.016309,0.079452
21309,03-Dec-2014,8537.65,0.001518,0.004722,0.016595,0.076712
22389,04-Dec-2014,8564.40,0.003128,0.004728,0.016921,0.073973
23469,05-Dec-2014,8538.30,-0.003052,0.004838,0.017992,0.068493
24549,08-Dec-2014,8438.25,-0.011787,0.005671,0.020295,0.073973


In [ ]:
option = df.merge(underlying, how='left', on='Date')

In [ ]:
option = option.dropna()

In [ ]:
option.head()

,Date,Strike Price,Close,Underlying Value,date_ndiff,stock_price,log_ret,sigma_20,cal_sig_20,delta_t
20229,02-Dec-2014,9550,1064.50,8524.7,22,8524.7,-0.003653,0.004723,0.016309,0.079452
20230,02-Dec-2014,9600,1918.40,8524.7,22,8524.7,-0.003653,0.004723,0.016309,0.079452
20231,02-Dec-2014,9650,1279.00,8524.7,22,8524.7,-0.003653,0.004723,0.016309,0.079452
20232,02-Dec-2014,9700,1952.65,8524.7,22,8524.7,-0.003653,0.004723,0.016309,0.079452
20233,02-Dec-2014,9750,1162.40,8524.7,22,8524.7,-0.003653,0.004723,0.016309,0.079452


In [ ]:
option = option.drop(columns=['Underlying Value'])
option.head()

,Date,Strike Price,Close,date_ndiff,stock_price,log_ret,sigma_20,cal_sig_20,delta_t
20229,02-Dec-2014,9550,1064.50,22,8524.7,-0.003653,0.004723,0.016309,0.079452
20230,02-Dec-2014,9600,1918.40,22,8524.7,-0.003653,0.004723,0.016309,0.079452
20231,02-Dec-2014,9650,1279.00,22,8524.7,-0.003653,0.004723,0.016309,0.079452
20232,02-Dec-2014,9700,1952.65,22,8524.7,-0.003653,0.004723,0.016309,0.079452
20233,02-Dec-2014,9750,1162.40,22,8524.7,-0.003653,0.004723,0.016309,0.079452


## Interest Rate
This step involved calculating interest rate from bond yields data

In [ ]:
option['date_ndiff'].max()

1826

In [ ]:
df1yr = pd.read_csv("India 1-Year Bond Yield Historical Data.csv")
df2yr = pd.read_csv("India 2-Year Bond Yield Historical Data.csv")
df3yr = pd.read_csv("India 3-Year Bond Yield Historical Data.csv")
df4yr = pd.read_csv("India 4-Year Bond Yield Historical Data.csv")
df5yr = pd.read_csv("India 5-Year Bond Yield Historical Data.csv")
df6yr = pd.read_csv("India 6-Year Bond Yield Historical Data.csv")
df3mo = pd.read_csv("India 3-Month Bond Yield Historical Data.csv") 
df6mo = pd.read_csv("India 6-Month Bond Yield Historical Data.csv") 

In [ ]:
df1yr = df1yr.rename(columns={'Open': '1yr'})
df1yr = df1yr[['Date', '1yr']]
df1yr.head()

,Date,1yr
0,"Dec 31, 2020",3.786
1,"Dec 30, 2020",3.773
2,"Dec 29, 2020",3.879
3,"Dec 28, 2020",3.802
4,"Dec 24, 2020",3.884


In [ ]:
df2yr = df2yr.rename(columns={'Open': '2yr'})
df2yr = df2yr[['Date', '2yr']]
df3yr = df3yr.rename(columns={'Open': '3yr'})
df3yr = df3yr[['Date', '3yr']]
df4yr = df4yr.rename(columns={'Open': '4yr'})
df4yr = df4yr[['Date', '4yr']]
df5yr = df5yr.rename(columns={'Open': '5yr'})
df5yr = df5yr[['Date', '5yr']]
df6yr = df6yr.rename(columns={'Open': '6yr'})
df6yr = df6yr[['Date', '6yr']]
df3mo = df3mo.rename(columns={'Open': '3mo'})
df3mo = df3mo[['Date', '3mo']]
df6mo = df6mo.rename(columns={'Open': '6mo'})
df6mo = df6mo[['Date', '6mo']]

In [ ]:
df_joined = df3mo.merge(df6mo, how='outer', on='Date')
df_joined.head()

,Date,3mo,6mo
0,"Dec 31, 2020",3.08,3.36
1,"Dec 30, 2020",3.11,3.36
2,"Dec 29, 2020",3.13,3.38
3,"Dec 28, 2020",3.13,3.37
4,"Dec 24, 2020",3.13,3.38


In [ ]:
len(df3mo)

2848

In [ ]:
df_joined = df_joined.merge(df1yr, how='outer', on='Date')
df_joined = df_joined.merge(df2yr, how='outer', on='Date')
df_joined = df_joined.merge(df3yr, how='outer', on='Date')
df_joined = df_joined.merge(df4yr, how='outer', on='Date')
df_joined = df_joined.merge(df5yr, how='outer', on='Date')
df_joined = df_joined.merge(df6yr, how='outer', on='Date')
df_joined.head()

,Date,3mo,6mo,1yr,2yr,3yr,4yr,5yr,6yr
0,"Dec 31, 2020",3.08,3.36,3.786,3.906,4.453,4.795,5.125,5.554
1,"Dec 30, 2020",3.11,3.36,3.773,3.910,4.498,4.818,5.135,5.590
2,"Dec 29, 2020",3.13,3.38,3.879,4.091,4.489,4.852,5.137,5.609
3,"Dec 28, 2020",3.13,3.37,3.802,3.888,4.461,4.825,5.137,5.609
4,"Dec 24, 2020",3.13,3.38,3.884,3.872,4.476,4.881,5.266,5.646


In [ ]:
len(df_joined)

2860

In [ ]:
df_joined['Date'] = df_joined.apply(lambda row: date_format_change(row), axis=1)

In [ ]:
df_joined.tail()

,Date,3mo,6mo,1yr,2yr,3yr,4yr,5yr,6yr
2855,09-Jun-2011,NaN,NaN,8.199,8.200,8.185,8.342,8.349,8.370
2856,08-Jun-2011,NaN,NaN,8.185,8.199,8.196,8.348,8.336,8.365
2857,07-Jun-2011,NaN,NaN,8.207,8.208,8.200,8.365,8.351,8.378
2858,06-Jun-2011,NaN,NaN,8.205,8.217,8.199,8.352,8.367,8.375
2859,16-Sep-2018,NaN,NaN,NaN,NaN,NaN,8.202,8.232,NaN


In [ ]:
combined_df = option.merge(df_joined, how='left', on='Date')
combined_df.head()

,Date,Strike Price,Close,date_ndiff,stock_price,log_ret,sigma_20,cal_sig_20,delta_t,3mo,6mo,1yr,2yr,3yr,4yr,5yr,6yr
0,02-Dec-2014,9550,1064.50,22,8524.7,-0.003653,0.004723,0.016309,0.079452,8.26,NaN,8.167,7.942,8.065,8.042,8.001,8.041
1,02-Dec-2014,9600,1918.40,22,8524.7,-0.003653,0.004723,0.016309,0.079452,8.26,NaN,8.167,7.942,8.065,8.042,8.001,8.041
2,02-Dec-2014,9650,1279.00,22,8524.7,-0.003653,0.004723,0.016309,0.079452,8.26,NaN,8.167,7.942,8.065,8.042,8.001,8.041
3,02-Dec-2014,9700,1952.65,22,8524.7,-0.003653,0.004723,0.016309,0.079452,8.26,NaN,8.167,7.942,8.065,8.042,8.001,8.041
4,02-Dec-2014,9750,1162.40,22,8524.7,-0.003653,0.004723,0.016309,0.079452,8.26,NaN,8.167,7.942,8.065,8.042,8.001,8.041


In [ ]:
combined_df['r'] = combined_df.apply(lambda row: get_interest_rate(row), axis=1)

In [ ]:
combined_df.tail()

,Date,Strike Price,Close,date_ndiff,stock_price,log_ret,sigma_20,cal_sig_20,delta_t,3mo,6mo,1yr,2yr,3yr,4yr,5yr,6yr,r
1503580,31-Dec-2020,9000,1425.55,1638,13981.75,-0.000014,0.024552,0.040835,0.342466,3.08,3.36,3.786,3.906,4.453,4.795,5.125,5.554,5.125
1503581,31-Dec-2020,9500,1621.40,1638,13981.75,-0.000014,0.024552,0.040835,0.342466,3.08,3.36,3.786,3.906,4.453,4.795,5.125,5.554,5.125
1503582,31-Dec-2020,10000,1827.45,1638,13981.75,-0.000014,0.024552,0.040835,0.342466,3.08,3.36,3.786,3.906,4.453,4.795,5.125,5.554,5.125
1503583,31-Dec-2020,8000,1066.65,1638,13981.75,-0.000014,0.024552,0.040835,0.342466,3.08,3.36,3.786,3.906,4.453,4.795,5.125,5.554,5.125
1503584,31-Dec-2020,15000,2300.00,1638,13981.75,-0.000014,0.024552,0.040835,0.342466,3.08,3.36,3.786,3.906,4.453,4.795,5.125,5.554,5.125


In [ ]:
combined_df = combined_df.drop(columns=['3mo', '6mo', '1yr', '2yr', '3yr', '4yr', '5yr', '6yr'], axis=1)

In [ ]:
combined_df['r'] = combined_df['r']/100 

In [ ]:
combined_df = combined_df.dropna()

## Time to maturity

In [ ]:
combined_df['tau'] = combined_df['date_ndiff']/365
combined_df = combined_df.drop(columns=['date_ndiff'])


## Historical 20-day data
This step involved calculating underlying price and return for for the past 20 days

In [ ]:
N_TIMESTEPS = 20
padded = np.insert(underlying['stock_price'].values, 0, np.array([np.nan] * N_TIMESTEPS))
rolled = np.column_stack([np.roll(padded, i) for i in range(N_TIMESTEPS)])
rolled = rolled[~np.isnan(rolled).any(axis=1)]
rolled = np.column_stack((underlying['Date'].values[N_TIMESTEPS - 1:], rolled))
price_history = pd.DataFrame(data=rolled, columns=['Date',]+['S' + str(i) for i in range(1, 21)])

In [ ]:
price_history.head()

,Date,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S20
0,30-Dec-2014,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7
1,31-Dec-2014,8282.7,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65
2,01-Jan-2015,8284,8282.7,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4
3,02-Jan-2015,8395.45,8284,8282.7,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3
4,05-Jan-2015,8378.4,8395.45,8284,8282.7,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25


In [ ]:
padded = np.insert(underlying['log_ret'].values, 0, np.array([np.nan] * N_TIMESTEPS))
rolled = np.column_stack([np.roll(padded, i) for i in range(N_TIMESTEPS)])
rolled = rolled[~np.isnan(rolled).any(axis=1)]
rolled = np.column_stack((underlying['Date'].values[N_TIMESTEPS - 1:], rolled))
return_history = pd.DataFrame(data=rolled, columns=['Date',]+['log_ret' + str(i) for i in range(1, 21)])

In [ ]:
final_df = combined_df.merge(price_history,how='left',on='Date')
final_df = final_df.merge(return_history,how='left',on='Date')

In [ ]:
final_df = final_df.rename(columns={'Strike Price':'K','Close':'option_price'})

In [ ]:
final_df = final_df.dropna()

In [ ]:
final_df.head()

,Date,K,option_price,stock_price,log_ret,sigma_20,cal_sig_20,delta_t,r,tau,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S20,log_ret1,log_ret2,log_ret3,log_ret4,log_ret5,log_ret6,log_ret7,log_ret8,log_ret9,log_ret10,log_ret11,log_ret12,log_ret13,log_ret14,log_ret15,log_ret16,log_ret17,log_ret18,log_ret19,log_ret20
19212,30-Dec-2014,10000,1662.75,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236442,0.0055451,0.0032489,-0.0113011,-0.00687122,0.0119403,0.00804423,0.0159988,-0.00469642,-0.0186655,-0.000547322,-0.00833086,-0.00753823,0.00179081,-0.0116278,-0.011787,-0.00305215,0.00312828,0.00151796,-0.00365327
19213,30-Dec-2014,9550,967.00,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236442,0.0055451,0.0032489,-0.0113011,-0.00687122,0.0119403,0.00804423,0.0159988,-0.00469642,-0.0186655,-0.000547322,-0.00833086,-0.00753823,0.00179081,-0.0116278,-0.011787,-0.00305215,0.00312828,0.00151796,-0.00365327
19214,30-Dec-2014,9600,1233.25,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236442,0.0055451,0.0032489,-0.0113011,-0.00687122,0.0119403,0.00804423,0.0159988,-0.00469642,-0.0186655,-0.000547322,-0.00833086,-0.00753823,0.00179081,-0.0116278,-0.011787,-0.00305215,0.00312828,0.00151796,-0.00365327
19215,30-Dec-2014,9650,1005.00,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236442,0.0055451,0.0032489,-0.0113011,-0.00687122,0.0119403,0.00804423,0.0159988,-0.00469642,-0.0186655,-0.000547322,-0.00833086,-0.00753823,0.00179081,-0.0116278,-0.011787,-0.00305215,0.00312828,0.00151796,-0.00365327
19216,30-Dec-2014,9700,1195.10,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267,8324,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236442,0.0055451,0.0032489,-0.0113011,-0.00687122,0.0119403,0.00804423,0.0159988,-0.00469642,-0.0186655,-0.000547322,-0.00833086,-0.00753823,0.00179081,-0.0116278,-0.011787,-0.00305215,0.00312828,0.00151796,-0.00365327


## Sharpe Ratio

In [ ]:
def sharpe_ratio(row):
    sig = row['sigma_20']
    avg_ret = np.mean([row['log_ret' + str(i)] for i in range(1, 21)])
    r = row['r']
    return (avg_ret - r)/sig

In [ ]:
final_df['sharpe_ratio'] = final_df.apply(lambda row: sharpe_ratio(row), axis=1)

## Moneyness

In [ ]:
def moneyness2(row):
    return (row['stock_price']*np.exp(row['r']*row['tau']))/row['K']

In [ ]:
final_df['moneyness1'] = final_df['stock_price']/final_df['K']
final_df['moneyness2'] = final_df.apply(lambda x: moneyness2(x), axis=1)

## log K/S features

In [ ]:
final_df['log(K/S)'] = np.log(final_df['K']/final_df['stock_price'])

In [ ]:
final_df['log(K/S)0.5'] = final_df['log(K/S)']*(final_df['tau']**(-0.5))

In [ ]:
final_df['log(K/S)0.95'] = final_df['log(K/S)']*(final_df['tau']**(-0.95))

## Final touch-up
This step involved removing absurd entries, dropping unnecessary columns, converting data from object to numerical, etc

In [ ]:
for col in final_df.columns:
  if col == 'Date':
    continue
  final_df[col] = pd.to_numeric(final_df[col], errors='coerce') 

In [ ]:
final_df = final_df.dropna()
final_df = final_df[final_df['tau']>0]
final_df = final_df[final_df['stock_price']>0]

In [ ]:
final_df.columns

Index(['Date', 'K', 'option_price', 'stock_price', 'log_ret', 'sigma_20',
       'cal_sig_20', 'delta_t', 'r', 'tau', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6',
       'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16',
       'S17', 'S18', 'S19', 'S20', 'log_ret1', 'log_ret2', 'log_ret3',
       'log_ret4', 'log_ret5', 'log_ret6', 'log_ret7', 'log_ret8', 'log_ret9',
       'log_ret10', 'log_ret11', 'log_ret12', 'log_ret13', 'log_ret14',
       'log_ret15', 'log_ret16', 'log_ret17', 'log_ret18', 'log_ret19',
       'log_ret20', 'sharpe_ratio', 'moneyness1', 'moneyness2', 'log(K/S)',
       'log(K/S)0.5', 'log(K/S)0.95'],
      dtype='object')

In [ ]:
final_df = final_df.dropna()

In [ ]:
final_df.head()

,Date,K,option_price,stock_price,log_ret,sigma_20,cal_sig_20,delta_t,r,tau,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S20,log_ret1,log_ret2,log_ret3,log_ret4,log_ret5,log_ret6,log_ret7,log_ret8,log_ret9,log_ret10,log_ret11,log_ret12,log_ret13,log_ret14,log_ret15,log_ret16,log_ret17,log_ret18,log_ret19,log_ret20,sharpe_ratio,moneyness1,moneyness2,log(K/S),log(K/S)0.5,log(K/S)0.95
19212,30-Dec-2014,10000,1662.75,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.824825,0.830505,0.192584,0.671748,2.067918
19213,30-Dec-2014,9550,967.00,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.863691,0.869639,0.146540,0.511143,1.573510
19214,30-Dec-2014,9600,1233.25,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.859193,0.865110,0.151762,0.529357,1.629582
19215,30-Dec-2014,9650,1005.00,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.854741,0.860627,0.156957,0.547477,1.685363
19216,30-Dec-2014,9700,1195.10,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.850335,0.856191,0.162125,0.565504,1.740855


In [ ]:
len(final_df)

19610

In [ ]:
final_df.to_csv('new_model_put_df.csv', index=True)

In [ ]:
df4matlab = final_df[['K', 'option_price', 'stock_price','r', 'tau']]

In [ ]:
df4matlab.head()

,K,option_price,stock_price,r,tau
19212,10000,1662.75,8248.25,0.0835,0.082192
19213,9550,967.00,8248.25,0.0835,0.082192
19214,9600,1233.25,8248.25,0.0835,0.082192
19215,9650,1005.00,8248.25,0.0835,0.082192
19216,9700,1195.10,8248.25,0.0835,0.082192


In [ ]:
df4matlab.to_csv('new_model_put_matlab.csv', index=True)

In [ ]:
## At this stage use the matlab code provided to calculate implied volatility for the option

# Implied Volatility

In [ ]:
final_df = pd.read_csv('new_model_put_df.csv')

In [ ]:
final_df = final_df.rename(columns={'Unnamed: 0': 'Index'})

In [ ]:
final_df.head()

,Index,Date,K,option_price,stock_price,log_ret,sigma_20,cal_sig_20,delta_t,r,tau,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S20,log_ret1,log_ret2,log_ret3,log_ret4,log_ret5,log_ret6,log_ret7,log_ret8,log_ret9,log_ret10,log_ret11,log_ret12,log_ret13,log_ret14,log_ret15,log_ret16,log_ret17,log_ret18,log_ret19,log_ret20,sharpe_ratio,moneyness1,moneyness2,log(K/S),log(K/S)0.5,log(K/S)0.95
0,19212,30-Dec-2014,10000,1662.75,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.824825,0.830505,0.192584,0.671748,2.067918
1,19213,30-Dec-2014,9550,967.00,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.863691,0.869639,0.146540,0.511143,1.573510
2,19214,30-Dec-2014,9600,1233.25,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.859193,0.865110,0.151762,0.529357,1.629582
3,19215,30-Dec-2014,9650,1005.00,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.854741,0.860627,0.156957,0.547477,1.685363
4,19216,30-Dec-2014,9700,1195.10,8248.25,0.000236,0.008876,0.031192,0.076712,0.0835,0.082192,8248.25,8246.3,8200.7,8174.1,8267.0,8324.0,8225.2,8159.3,8029.8,8067.6,8219.6,8224.1,8292.9,8355.65,8340.7,8438.25,8538.3,8564.4,8537.65,8524.7,0.000236,0.005545,0.003249,-0.011301,-0.006871,0.01194,0.008044,0.015999,-0.004696,-0.018666,-0.000547,-0.008331,-0.007538,0.001791,-0.011628,-0.011787,-0.003052,0.003128,0.001518,-0.003653,-9.613802,0.850335,0.856191,0.162125,0.565504,1.740855


In [ ]:
impl_df = pd.read_csv("new_model_put_impl_sig.csv")

In [ ]:
impl_df.head()

In [ ]:
final_df = final_df.merge(impl_df.drop(columns=['K', 'option_price', 'stock_price', 'r', 'tau']),on=['Index'],how='left')

In [ ]:
for col in final_df.columns:
  if col == 'Date':
    continue
  final_df[col] = pd.to_numeric(final_df[col], errors='coerce') 
final_df = final_df.dropna()

In [ ]:
final_df.head()

# Evicting Outliers

## Removing deep in-the-money and out-of-money

In [ ]:
final_df = final_df[final_df['moneyness2']<=1.10]
final_df = final_df[final_df['stock_price']>=0.90]

In [ ]:
len(final_df)

## Removing Arbitrage Opportunities

In [ ]:
final_df.columns

In [ ]:
def arbitrage(row):
    return row['option_price'] - max(row['stock_price']-row['K']*np.exp(-row['r']*row['tau']), 0)

In [ ]:
final_df['arbitrage'] = final_df[['stock_price', 'r', 'K', 'tau', 'option_price']].apply(lambda x: arbitrage(x), axis=1)

In [ ]:
final_df = final_df[final_df['arbitrage']>0]

In [ ]:
len(final_df)

## Expiry

In [ ]:
final_df = final_df[final_df['tau']>=6/365]

In [ ]:
len(final_df)

In [ ]:
final_df.columns

# Final data csv

In [ ]:
final_df = final_df[['K', 'stock_price', 
                     'log_ret', 'sigma_20', 'cal_sig_20', 'delta_t', 'r', 
                     'tau', 'sharpe_ratio', 'moneyness1', 'moneyness2', 
                     'log(K/S)', 'log(K/S)0.5', 'log(K/S)0.95', 'arbitrage', 
                     'S1', 'log_ret1', 'S2', 'log_ret2', 'S3', 'log_ret3', 
                     'S4', 'log_ret4', 'S5', 'log_ret5', 'S6', 'log_ret6',
                     'S7', 'log_ret7', 'S8', 'log_ret8', 'S9', 'log_ret9', 
                     'S10', 'log_ret10', 'S11', 'log_ret11', 'S12', 'log_ret12', 
                     'S13', 'log_ret13', 'S14', 'log_ret14', 'S15', 'log_ret15', 
                     'S16', 'log_ret16', 'S17', 'log_ret17', 'S18', 'log_ret18', 
                     'S19', 'log_ret19', 'S20', 'log_ret20', 'impl_sig', 'option_price']]

In [ ]:
final_df.columns

In [ ]:
final_df.to_csv('new_model_final_put_df.csv', index=False)